In [1]:
import json
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, timedelta


In [2]:
df = pd.DataFrame([], columns = ['PubDate', 'URL', 'byline', 'headline', 'standfirst', 'body'])


In [3]:

start_date = date(2020, 4, 1)
end_date = date(2020, 10, 31)
dayrange = range((end_date - start_date).days + 1)
for daycount in dayrange:
    dt = start_date + timedelta(days=daycount)
    datestr = dt.strftime('%Y-%m-%d')
    fname = 'articles/' + datestr + '.json'
#     print(fname)

    with open(fname) as f:
        data = json.load(f)

    for article in data:

        if 'webPublicationDate' in article: 
            pub_date = article['webPublicationDate']
        else:
            pub_date = ""

        if 'webUrl' in article: 
            URL = article['webUrl']
        else:
            URL = ""
        
        if 'headline' in article['fields']: 
            headline = article['fields']['headline']
        else:
            headline = ""
            
        if 'standfirst' in article['fields']: 
            standfirst_soup = BeautifulSoup(article['fields']['standfirst'], features='html.parser')
            standfirst = standfirst_soup.get_text()
        else:
            standfirst = ""
                                         
        if 'byline' in article['fields']: 
            byline = article['fields']['byline']
        else:
            byline = ""

        if 'body' in article['fields']: 
            body_soup = BeautifulSoup(article['fields']['body'], features='html.parser')
            body = body_soup.get_text()
        else:
            body = ""

        new_record = {
            'PubDate': pub_date,
            'URL': URL,
            'headline': headline,
            'standfirst': standfirst,
            'byline': byline,
            'body': body,
        }

        df.loc[len(df.index)] = new_record


In [4]:
df.shape

(3067, 6)

In [6]:
def is_rumour_mill(row):
    return row['URL'].find('transfer-rumours') >= 0 

In [7]:
df['is_rumour_mill'] = df.apply(is_rumour_mill, axis=1)

In [8]:
df.shape

(3067, 7)

In [9]:
df.head()

,PubDate,URL,byline,headline,standfirst,body,is_rumour_mill
0,2020-04-01T20:16:54Z,https://www.theguardian.com/football/2020/apr/...,David Conn,Premier League tells PFA players will have to ...,Deferrals of wages not cuts wanted by football...,The Premier League and EFL have urged the foot...,False
1,2020-04-01T18:44:06Z,https://www.theguardian.com/football/blog/2020...,Paul MacInnes,Premier League clubs fail the smell test by fu...,Belts are being tightened across the UK but it...,Under the shadow of coronavirus new rules are ...,False
2,2020-04-01T15:57:46Z,https://www.theguardian.com/football/2020/apr/...,Simon Burnton,"The Premier League, pay cuts and the plight of...",Sign up now! Sign up now! Sign up now? Sign up...,HOT SHIITAKE There was consternation among Pre...,False
3,2020-04-01T15:13:23Z,https://www.theguardian.com/football/2020/apr/...,Suzanne Wrack,Women's Euro 2021 in England postponed by a ye...,Move widely anticipated amid coronavirus pande...,"The women’s European Championship, which was d...",False
4,2020-04-01T14:48:34Z,https://www.theguardian.com/football/2020/apr/...,Ben Fisher,Bournemouth manager Eddie Howe takes 'signific...,Three other senior employees doing the sameBou...,Eddie Howe has become the first Premier League...,False
